In [61]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [62]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [63]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough

In [64]:
# 텍스트로부터 FAISS 벡터 저장소를 생성합니다.
vectorstore = FAISS.from_texts(
    [
        "인환은 소놀코딩에서 근무를 하였습니다.",
        "윤정은 인환과 같은 회사에서 근무하였습니다.",
        "인환의 직업은 개발자입니다.",
        "윤정의 직업은 디자이너입니다.",
    ],
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

In [65]:
# 벡터 저장소를 검색기로 사용합니다.
retriever = vectorstore.as_retriever()

In [66]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [67]:
prompt = ChatPromptTemplate.from_template(template)

In [68]:
outputparser = StrOutputParser()

In [69]:
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [70]:
# 문서를 포맷팅하는 함수
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [71]:
# def format_docs(docs):
#     formatted_contents = []

#     for doc in docs:
#         formatted_contents.append(doc.page_content)
        
#     return "\n".join(formatted_contents)

In [72]:
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | outputparser
)

In [73]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("인환의 직업은 무엇입니까?")

'인환의 직업은 개발자입니다.'

In [74]:
# 검색 체인을 실행하여 질문에 대한 답변을 얻습니다.
retrieval_chain.invoke("윤정의 직업은 무엇입니까?")

'윤정의 직업은 디자이너입니다.'